In [1]:
!pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install torch torchvision torchaudio


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Load tokenized datasets directly from April–May notebook output
import pickle

with open('datasets/security/train_tokenized.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

with open('datasets/security/valid_tokenized.pkl', 'rb') as f:
    valid_dataset = pickle.load(f)

with open('datasets/security/test_tokenized.pkl', 'rb') as f:
   test_dataset = pickle.load(f)

In [5]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install --upgrade transformers


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd

train_df = pd.read_pickle("datasets/security/train_tokenized.pkl")
valid_df = pd.read_pickle("datasets/security/valid_tokenized.pkl")

In [15]:
print(train_df.columns)
print(train_df.head())

Index(['id', 'func', 'target', 'project', 'commit_id', 'func_length',
       'tokens'],
      dtype='object')
   id                                               func  target project  \
0   0  static av_cold int vdadec_init(AVCodecContext ...   False  FFmpeg   
1   1  static int transcode(AVFormatContext **output_...   False  FFmpeg   
2   2  static void v4l2_free_buffer(void *opaque, uin...   False  FFmpeg   
3   4  int av_opencl_buffer_write(cl_mem dst_cl_buf, ...   False  FFmpeg   
4   5  static int r3d_read_rdvo(AVFormatContext *s, A...    True  FFmpeg   

                                  commit_id  func_length  \
0  973b1a6b9070e2bf17d17568cbaf4043ce931f51          191   
1  321b2a9ded0468670b7678b7c098886930ae16b2         2687   
2  5d5de3eba4c7890c2e8077f5b4ae569671d11cf8           38   
3  57d77b3963ce1023eaf5ada8cba58b9379405cc8           72   
4  aba232cfa9b193604ed98f3fa505378d006b1b3b           74   

                        tokens  
0  [input_ids, attention_mask]  
1  [in

In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert DataFrame columns to tensors
train_data = TensorDataset(
    torch.tensor(train_df["input_ids"].tolist(), dtype=torch.long),
    torch.tensor(train_df["attention_mask"].tolist(), dtype=torch.long),
    torch.tensor(train_df["labels"].tolist(), dtype=torch.long)
)

valid_data = TensorDataset(
    torch.tensor(valid_df["input_ids"].tolist(), dtype=torch.long),
    torch.tensor(valid_df["attention_mask"].tolist(), dtype=torch.long),
    torch.tensor(valid_df["labels"].tolist(), dtype=torch.long)
)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=8)

KeyError: 'input_ids'

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, output_dict=True)
    return {
        "accuracy": report["accuracy"],
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"],
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
trainer.evaluate()